In [2]:
import pandas as pd
import os
import json
from datetime import datetime
from tqdm import tqdm
import math

In [12]:
def get_all_files_path_under(folder_path):
    
    all_jsons = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            all_jsons.append( os.path.normpath (file_path) )
    return all_jsons

In [13]:
from dateutil import parser
def time_difference(datetime_str1,datetime_str2):
    # Define your two datetime strings
    datetime_str1 = '2017-04-12T23:00:00Z'
    datetime_str2 = '2017-04-13T01:41:46Z'

    # Parse the datetime strings into datetime objects
    datetime_obj1 = parser.parse(datetime_str1)
    datetime_obj2 = parser.parse(datetime_str2)

    # Calculate the time difference
    time_difference = datetime_obj2 - datetime_obj1

    # Extract hours and minutes from the time difference
    hours, remainder = divmod(time_difference.total_seconds(), 3600)
    minutes = remainder / 60

    # Print the result
    return f"{int(hours)} hours and {int(minutes)} minutes"

In [14]:
def get_periodes(data):
    periods = data.get('liveData').get('linescore').get('periods')
    my_dict = {i: None for i in range(1,len(periods)+1)}
    for i in range(len(periods)):
        my_dict[i+1] = {'home': periods[i].get('home').get('rinkSide'), 'away':periods[i].get('away').get('rinkSide') }
    return my_dict

In [15]:
def home_or_away(shooterTeam,home,away):
    if shooterTeam == home:
        return str('home')
    if shooterTeam == away:
        return str('away')

In [16]:
def get_distance(eventPeriod,home_away,x,y,rink_info):
        if eventPeriod == 5:
            return 'shoot out'
        if home_away == 'home':
            rinkSide = rink_info[eventPeriod]['home']
            if rinkSide == 'right':
                return  math.sqrt((x +100)**2 + (y)**2)
            if rinkSide == 'left':
                return  math.sqrt((x -100)**2 + (y)**2)
        if home_away == 'away':
            rinkSide = rink_info[eventPeriod]['away']
            if rinkSide == 'right':
                return  math.sqrt((x +100)**2 + (y)**2)
            if rinkSide == 'left':
                return  math.sqrt((x -100)**2 + (y)**2)

In [17]:
def get_angle(eventPeriod,home_away,x,y,rink_info):
    if eventPeriod == 5:
        return 'shoot out'
    if home_away == 'home':
        rinkSide = rink_info[eventPeriod]['home']
        if rinkSide == 'right':
            a = abs(y)
            b = math.sqrt((x + 100)**2 + (y)**2)
            c = abs(x + 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))
        if rinkSide == 'left':
            a = abs(y)
            b = math.sqrt((x -100)**2 + (y)**2)
            c = abs(x - 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))
    if home_away == 'away':
        rinkSide = rink_info[eventPeriod]['away']
        if rinkSide == 'right':
            a = abs(y)
            b = math.sqrt((x + 100)**2 + (y)**2)
            c = abs(x + 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))
        if rinkSide == 'left':
            a = abs(y)
            b = math.sqrt((x -100)**2 + (y)**2)
            c = abs(x - 100)
            return math.degrees(math.acos((b**2 + c**2 - a**2)/(2*b*c)))

In [8]:
data_folder = 'data/'
folder_path_20162017P = data_folder + '2016-2017/P'
folder_path_20162017R = data_folder + '2016-2017/R'

In [9]:
folder_path_20172018P = data_folder + '2017-2018/P'
folder_path_20172018R = data_folder + '2017-2018/R'

In [10]:
folder_path_20182019P = data_folder  + '2018-2019/P'
folder_path_20182019R = data_folder  + '2018-2019/R'

In [11]:
folder_path_20192020P = data_folder  + '2019-2020/P'
folder_path_20192020R = data_folder  + '2019-2020/R'

In [12]:
folder_path_20202021P = data_folder  + '2020-2021/P'
folder_path_20202021R = data_folder  + '2020-2021/R'

In [13]:
def create_csv(folder_path):
    data_to_tidy = get_all_files_path_under(folder_path)
    #columns = ['gamePk','season','gameType','home','away',
 #'eventTypeId','shotType','shooter','goalie','eventPeriod','eventPeriodType','x','y'

#]
    #df = dd.from_pandas(pd.DataFrame(columns = columns), npartitions=1)
    data_list = []
    for j in tqdm(range(len(data_to_tidy))):
        json_file = data_to_tidy[j]

        with open(json_file) as json_file:
            data = json.load(json_file)

        gamePk = data.get('gamePk')

        gameData = data.get('gameData')
        game = gameData.get('game')
        season = game.get('season')
        gameType = game.get('type')

        teams = gameData.get('teams')
        home =  teams.get('home').get('name')
        away =  teams.get('away').get('name')
        
        rink_info = get_periodes(data)
        plays = data.get('liveData').get('plays').get('allPlays')
     
        for i in range(len(plays)):

            play = plays[i]
            about= play.get('about')
            res = play.get('result')
            eventTypeId = res.get('eventTypeId')
            eventPeriod = about.get('period')
            eventPeriodType = about.get('periodType')
            
       

            shotType = shooter = goalie =shooterTeam = shotResult= distance = angle = None
            isGoal = 0
            emptyNet = 0
            players = play.get('players')
              
            players = play.get('players')
            if eventTypeId == 'SHOT':
                shotType =  res.get('secondaryType')
                shooterTeam = play.get('team').get('name') 
                for player in players:
                    if player.get('playerType') == "Shooter":
                        shooter = player.get('player').get('fullName')
                    if  player.get('playerType') == "Goalie":
                        goalie = player.get('player').get('fullName')

            if eventTypeId == 'GOAL':
                shotType =  res.get('secondaryType') 
                shooterTeam = play.get('team').get('name') 
                isGoal = 1
                if res.get('emptyNet') == True:
                    emptyNet = 1
                for player in players:
                    if player.get('playerType') == 'Scorer':
                        shooter = player.get('player').get('fullName')
                    if  player.get('playerType') == "Goalie":
                        goalie = player.get('player').get('fullName') +'(failed)'

            if play.get('coordinates') == {}:
                            x = None
                            y = None
            else:
                            for coord in play.get('coordinates'):
                                if coord == 'x':
                                     x = play.get('coordinates')['x']
                                if coord == 'y':
                                     y = play.get('coordinates')['y']
            if shooterTeam is not None:  
                 if play.get('coordinates') != {}:
                    home_away = home_or_away(shooterTeam,home,away)   
                    distance =  get_distance(eventPeriod,home_away,x,y,rink_info)
                    angle = get_angle(eventPeriod,home_away,x,y,rink_info)
                    

            new_row = {'gamePk':gamePk,'season':season,'gameType':gameType,'home':home,'away':away,
                        'eventTypeId':eventTypeId,'shotType':shotType,
                       'shooter':shooter,'shooterTeam':shooterTeam,'distance':distance ,'angle':angle,'goalie':goalie,
                       'eventPeriod':eventPeriod,
                       'eventPeriodType':eventPeriodType,'x':x,'y':y,'emptyNet':emptyNet,'isGoal':isGoal

                      }
            
            #df_to_concat = dd.from_pandas(pd.DataFrame(new_row,index=index)， npartitions=2)
            #df = dd.concat([df, df_to_concat])
            data_list.append(new_row)
    df = pd.DataFrame(data_list)        
    path =os.path.normpath(folder_path) 
    path.split('/')[-2:]
    csv_name = path.split('/')[-2]+'_'+ path.split('/')[-1]+'_clean'+'.csv'
    df.to_csv(csv_name, encoding='utf-8', index=False)

In [14]:
create_csv(folder_path_20162017P)

100%|██████████| 87/87 [00:00<00:00, 129.93it/s]


In [15]:
create_csv(folder_path_20162017R)

100%|██████████| 1230/1230 [00:08<00:00, 150.30it/s]


In [16]:
create_csv(folder_path_20172018P)

100%|██████████| 84/84 [00:00<00:00, 137.79it/s]


In [17]:
create_csv(folder_path_20172018R)

100%|██████████| 1271/1271 [00:08<00:00, 144.85it/s]


In [18]:
create_csv(folder_path_20182019P)

100%|██████████| 87/87 [00:00<00:00, 131.59it/s]


In [19]:
create_csv(folder_path_20182019R)

100%|██████████| 1271/1271 [00:08<00:00, 147.71it/s]


In [20]:
create_csv(folder_path_20192020P)

100%|██████████| 86/86 [00:00<00:00, 125.87it/s]


In [21]:
create_csv(folder_path_20192020R)

100%|██████████| 1082/1082 [00:08<00:00, 130.26it/s]


In [22]:
df20162017R = pd.read_csv('2016-2017_R_clean.csv')
df20162017P = pd.read_csv('2016-2017_P_clean.csv')
df20162017 = pd.concat([df20162017R,df20162017P])
df20162017.to_csv('20162017_clean.csv', encoding='utf-8', index=False)

In [23]:
df20172018R = pd.read_csv('2017-2018_R_clean.csv')
df20172018P = pd.read_csv('2017-2018_P_clean.csv')
df20172018 = pd.concat([df20172018R,df20172018P])
df20172018.to_csv('20172018_clean.csv', encoding='utf-8', index=False)

In [24]:
df20182019R = pd.read_csv('2018-2019_R_clean.csv').sort_values(by='gamePk', ascending=True)
df20182019P = pd.read_csv('2018-2019_P_clean.csv').sort_values(by='gamePk', ascending=True)
df20182019 = pd.concat([df20182019R,df20182019P]).sort_values(by='gamePk', ascending=True)
df20182019.to_csv('20182019_clean.csv', encoding='utf-8', index=False)

In [25]:
df20192020R = pd.read_csv('2019-2020_R_clean.csv').sort_values(by='gamePk', ascending=True)
df20192020P = pd.read_csv('2019-2020_P_clean.csv').sort_values(by='gamePk', ascending=True)
df20192020 = pd.concat([df20192020R,df20192020P]).sort_values(by='gamePk', ascending=True)
df20192020.to_csv('20192020_clean.csv', encoding='utf-8', index=False)

In [26]:
df20162018 = pd.concat([df20162017 ,df20172018]).sort_values(by='gamePk', ascending=True)
df20162019 = pd.concat([df20162018 ,df20182019]).sort_values(by='gamePk', ascending=True)
df20162019 = pd.concat([df20162019 ,df20192020]).sort_values(by='gamePk', ascending=True)
df20162019.to_csv('20162019_clean.csv', encoding='utf-8', index=False)

In [19]:
df20162019 = pd.read_csv('20162019_clean.csv')

In [21]:
df20162019_shot_goal = df20162019.loc[~df20162019['shooter'].isnull() & ~df20162019['shotType'].isnull()]
df20162019_shot_goal["eventTypeId"].unique()

array(['SHOT', 'GOAL'], dtype=object)

In [28]:
# df20162019_shooter = df20162019.loc[~df20162019['shooter'].isnull()]
# df20162019_shooter_nonNull = df20162019_shooter[~df20162019_shooter['shotType'].isnull()]
# shotType = df20162019_shooter_nonNull['shotType'].unique()
# eventType = df20162019_shooter_nonNull['eventTypeId'].unique()

In [31]:
# all "shooters" are "non-null", but "shotTypes have "nan"
# df20162019_shot_goal = df20162019[(df20162019["eventTypeId"]=="SHOT") | (df20162019["eventTypeId"]=="GOAL")]
# df20162019_goal = df20162019[df20162019["eventTypeId"]=="GOAL"]

In [3]:
df20162017 = pd.read_csv('data_after_fe2/20162017_clean.csv')
df20162017

,gamePk,season,gameType,home,away,eventTypeId,eventType,shotType,shooter,shooterTeam,...,eventID,prevEventID,timeFromLastEvent,prevEventX,prevEventY,prevEventType,distFromPrev,isRebound,changeInAngle,speed
0,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,GAME_SCHEDULED,Game Scheduled,NaN,NaN,NaN,...,1,664,-3600,NaN,NaN,GAME_END,NaN,False,0.0,NaN
1,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,PERIOD_READY,Period Ready,NaN,NaN,NaN,...,2,1,0,NaN,NaN,GAME_SCHEDULED,NaN,False,0.0,NaN
2,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,PERIOD_START,Period Start,NaN,NaN,NaN,...,51,2,0,NaN,NaN,PERIOD_READY,NaN,False,0.0,NaN
3,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,FACEOFF,Faceoff,NaN,NaN,NaN,...,52,51,0,NaN,NaN,PERIOD_START,NaN,False,0.0,NaN
4,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,SHOT,Shot,Wrist Shot,Kevin Shattenkirk,St. Louis Blues,...,53,52,33,0.0,0.0,FACEOFF,50.537115,False,0.0,1.531428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423668,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,PENALTY,Penalty,NaN,NaN,NaN,...,642,640,0,NaN,NaN,PENALTY,NaN,False,0.0,NaN
423669,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,FACEOFF,Faceoff,NaN,NaN,NaN,...,754,642,0,NaN,NaN,PENALTY,NaN,False,0.0,NaN
423670,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,PERIOD_END,Period End,NaN,NaN,NaN,...,644,754,34,69.0,-22.0,FACEOFF,NaN,False,0.0,NaN
423671,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,PERIOD_OFFICIAL,Period Official,NaN,NaN,NaN,...,649,644,0,NaN,NaN,PERIOD_END,NaN,False,0.0,NaN


In [4]:
df20162017.columns

Index(['gamePk', 'season', 'gameType', 'home', 'away', 'eventTypeId',
       'eventType', 'shotType', 'shooter', 'shooterTeam', 'distance', 'angle',
       'goalie', 'eventPeriod', 'eventPeriodType', 'x', 'y', 'emptyNet',
       'isGoal', 'game_time', 'eventID', 'prevEventID', 'timeFromLastEvent',
       'prevEventX', 'prevEventY', 'prevEventType', 'distFromPrev',
       'isRebound', 'changeInAngle', 'speed'],
      dtype='object')

In [7]:
df20162017["game_time"].describe()


count    423673.000000
mean       1810.722170
std        1082.293083
min           0.000000
25%         896.000000
50%        1788.000000
75%        2701.000000
max        5513.000000
Name: game_time, dtype: float64

In [11]:
df20162017_shot_goal = df20162017.loc[~df20162017['shooter'].isnull() & ~df20162017['shotType'].isnull()]
df20162017_shot_goal

,gamePk,season,gameType,home,away,eventTypeId,eventType,shotType,shooter,shooterTeam,...,eventID,prevEventID,timeFromLastEvent,prevEventX,prevEventY,prevEventType,distFromPrev,isRebound,changeInAngle,speed
4,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,SHOT,Shot,Wrist Shot,Kevin Shattenkirk,St. Louis Blues,...,53,52,33,0.0,0.0,FACEOFF,50.537115,False,0.000000,1.531428
13,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,SHOT,Shot,Wrist Shot,Petteri Lindbohm,St. Louis Blues,...,57,9,4,-92.0,36.0,HIT,154.207652,False,0.000000,38.551913
15,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,SHOT,Shot,Backhand,Zemgus Girgensons,Buffalo Sabres,...,58,10,1,22.0,-39.0,HIT,110.494344,False,0.000000,110.494344
19,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,SHOT,Shot,Slap Shot,Colton Parayko,St. Louis Blues,...,61,60,54,20.0,-22.0,FACEOFF,21.000000,False,0.000000,0.388889
22,2016020236,20162017,R,St. Louis Blues,Buffalo Sabres,SHOT,Shot,Wrist Shot,Petteri Lindbohm,St. Louis Blues,...,63,62,2,69.0,-22.0,FACEOFF,22.671568,False,0.000000,11.335784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423647,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,SHOT,Shot,Wrist Shot,P.K. Subban,Nashville Predators,...,587,586,95,61.0,2.0,MISSED_SHOT,106.254412,False,0.000000,1.118467
423651,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,SHOT,Shot,Wrist Shot,Frederick Gaudreau,Nashville Predators,...,591,901,6,-51.0,35.0,HIT,30.149627,False,0.000000,5.024938
423654,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,SHOT,Shot,Snap Shot,Craig Smith,Nashville Predators,...,594,593,8,-74.0,10.0,MISSED_SHOT,39.204592,False,0.000000,4.900574
423655,2016030415,20162017,P,Pittsburgh Penguins,Nashville Predators,SHOT,Shot,Slap Shot,P.K. Subban,Nashville Predators,...,595,594,5,-70.0,-29.0,SHOT,70.213959,True,14.894686,14.042792
